In [ ]:
files = ['/Users/alextaim/data/ccb/original/01.json']

In [ ]:
from VwPipeline import Loggers, Handlers
from VwPipeline.VwCache import VwCache
from VwPipeline.Vw import Vw
from VwPipeline.VwOpts import dimension, product
import pandas as pd

#your vw path
vw_path = r'vw'

cache = VwCache(r'/Users/alextaim/data/.vw_cache')
vw = Vw(
    vw_path,
    cache,
    handlers=[Handlers.WidgetHandler()],
    )

opts = pd.DataFrame(product(
    dimension('#base', ['--ccb_explore_adf --dsjson --compressed --synthcover --power_t 0  -P 1 --preserve_performance_counters --save_resume']),
))
preds = vw.train(files, opts, ['-p'])
prediction_files = preds.iloc[0]['!Outputs']['-p']
prediction_files

In [ ]:
def ccb_predictions(files, index = 't'):
    for f in files:
        for l in open(f):


class CcbPredictions:
    def __init__(self, files, index='t'):
        self.files = files
        self.index = index
        
    def iter_slots(self):
        result = {}
        for f in files:
            for l in open(f):
                o = json.loads(l)
                for 
                    yield result

# Usage

## Generate slim dsjson

In [ ]:
from Pipeline.ccb.dsjson import processor

In [ ]:
from Pipeline.dataflow import FilesPipeline
from Pipeline.ccb.dsjson.processor import Processor

processor = Processor()
pipeline = FilesPipeline()

slim = pipeline.lines_2_lines(files, processor.process, path_gen=lambda p: p.replace('original', 'slim'), process=True)
slim

## Sample

In [ ]:
from Pipeline.dataflow import FilesPipeline
from Pipeline.ccb.dsjson.processor import Processor
from Pipeline.filters import UniformSampler

sampler = UniformSampler(0.5)

processor = Processor(filters=[lambda l: sampler.do(l)])
result = pipeline.lines_2_lines(files, processor.process, path_gen=lambda p: p.replace('original', 'sample'), process=True)
result

## Predict

In [ ]:
from Pipeline.dataflow import FilesPipeline
from Pipeline.ccb.dsjson.predictions import Predictor

predictor = Predictor(filters=[lambda l: True])
cfe = pipeline.lines_2_lines(slim, predictor.predict, path_gen=lambda p: p.replace('slim', 'predict'), process=True)
cfe

# Preestimate

In [ ]:
from Pipeline.estimators import Estimator, evaluate
import Pipeline.ccb.estimators
import json

estimator = Estimator(factory = Pipeline.ccb.estimators.create, estimators = {'baseline1_old': ['ccb|snips|0'], 'random': ['ccb|snips|0']}, online_estimator = 'ccb|ips|0')
result = estimator.preestimate(map(lambda l: json.loads(l), open(cfe[0])), '1min')

# Evaluate

In [ ]:
evaluate(estimator.read_preestimate('test.csv'))

In [ ]:
import itertools
import json
predictions = map(lambda l: json.loads(l), itertools.chain.from_iterable([open(f) for f in cfe]))